<a href="https://colab.research.google.com/github/lalitha-sahitya/movie-review-classification/blob/main/imdb_reviews(single_LSTM).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow_datasets as tfds

In [2]:
imdb, info=tfds.load("imdb_reviews", with_info=True, as_supervised=True)

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Generating splits...:   0%|          | 0/3 [00:00<?, ? splits/s]

Generating train examples...: 0 examples [00:00, ? examples/s]

Shuffling /root/tensorflow_datasets/imdb_reviews/plain_text/incomplete.15P9IY_1.0.0/imdb_reviews-train.tfrecor…

Generating test examples...: 0 examples [00:00, ? examples/s]

Shuffling /root/tensorflow_datasets/imdb_reviews/plain_text/incomplete.15P9IY_1.0.0/imdb_reviews-test.tfrecord…

Generating unsupervised examples...: 0 examples [00:00, ? examples/s]

Shuffling /root/tensorflow_datasets/imdb_reviews/plain_text/incomplete.15P9IY_1.0.0/imdb_reviews-unsupervised.…

Dataset imdb_reviews downloaded and prepared to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0. Subsequent calls will reuse this data.


In [3]:
imdb

{Split('train'): <_PrefetchDataset element_spec=(TensorSpec(shape=(), dtype=tf.string, name=None), TensorSpec(shape=(), dtype=tf.int64, name=None))>,
 Split('test'): <_PrefetchDataset element_spec=(TensorSpec(shape=(), dtype=tf.string, name=None), TensorSpec(shape=(), dtype=tf.int64, name=None))>,
 Split('unsupervised'): <_PrefetchDataset element_spec=(TensorSpec(shape=(), dtype=tf.string, name=None), TensorSpec(shape=(), dtype=tf.int64, name=None))>}

In [4]:
train_data, test_data= imdb['train'], imdb['test']

In [6]:
for i in train_data.take(4):
  print(i)

(<tf.Tensor: shape=(), dtype=string, numpy=b"This was an absolutely terrible movie. Don't be lured in by Christopher Walken or Michael Ironside. Both are great actors, but this must simply be their worst role in history. Even their great acting could not redeem this movie's ridiculous storyline. This movie is an early nineties US propaganda piece. The most pathetic scenes were those when the Columbian rebels were making their cases for revolutions. Maria Conchita Alonso appeared phony, and her pseudo-love affair with Walken was nothing but a pathetic emotional plug in a movie that was devoid of any real meaning. I am disappointed that there are movies like this, ruining actor's like Christopher Walken's good name. I could barely sit through it.">, <tf.Tensor: shape=(), dtype=int64, numpy=0>)
(<tf.Tensor: shape=(), dtype=string, numpy=b'I have been known to fall asleep during films, but this is usually due to a combination of things including, really tired, being warm and comfortable on

In [7]:
train_sentences=[]
train_labels=[]
test_sequences=[]
test_labels=[]

In [8]:
for s,l in train_data:
  train_sentences.append(s.numpy().decode('utf8'))
  train_labels.append(l.numpy())
for s,l in test_data:
  test_sequences.append(s.numpy().decode('utf8'))
  test_labels.append(l.numpy())

In [10]:
train_sentences[0]

"This was an absolutely terrible movie. Don't be lured in by Christopher Walken or Michael Ironside. Both are great actors, but this must simply be their worst role in history. Even their great acting could not redeem this movie's ridiculous storyline. This movie is an early nineties US propaganda piece. The most pathetic scenes were those when the Columbian rebels were making their cases for revolutions. Maria Conchita Alonso appeared phony, and her pseudo-love affair with Walken was nothing but a pathetic emotional plug in a movie that was devoid of any real meaning. I am disappointed that there are movies like this, ruining actor's like Christopher Walken's good name. I could barely sit through it."

In [19]:
vocab_size=10000
embedding_dim=16
max_len=120
padding_type='pre'
trunc_type='post'

In [11]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [14]:
tokenizer=Tokenizer(num_words=10000, oov_token="<OOV>")

In [17]:
tokenizer.fit_on_texts(train_sentences)

In [18]:
word_index=tokenizer.word_index

In [20]:
sequences=tokenizer.texts_to_sequences(train_sentences)
padded_seq=pad_sequences(sequences, maxlen=max_len, padding=padding_type, truncating=trunc_type)

In [22]:
test_sequences=tokenizer.texts_to_sequences(test_sequences)
padded_test_seq=pad_sequences(test_sequences, maxlen=max_len, padding=padding_type, truncating=trunc_type)

In [36]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(padded_seq, train_labels, test_size=0.2, random_state=42)

In [37]:
embedding_dim=64
dense_dim=64
lstm_dim=64

In [38]:
model=tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_len),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(lstm_dim)),
    tf.keras.layers.Dense(dense_dim, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [39]:
model.summary()

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_2 (Embedding)         │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional_2 (Bidirectional) │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [40]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [45]:
import numpy as np

y_train = np.array(y_train)
y_test = np.array(y_test)

In [46]:
history=model.fit(x_train, y_train, epochs=10, validation_data=(x_test, y_test))

Epoch 1/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 96s 144ms/step - accuracy: 0.6913 - loss: 0.5565 - val_accuracy: 0.8338 - val_loss: 0.3694
Epoch 2/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 136s 135ms/step - accuracy: 0.8879 - loss: 0.2809 - val_accuracy: 0.8362 - val_loss: 0.3752
Epoch 3/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 144s 138ms/step - accuracy: 0.9277 - loss: 0.1875 - val_accuracy: 0.8306 - val_loss: 0.4772
Epoch 4/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 87s 139ms/step - accuracy: 0.9606 - loss: 0.1104 - val_accuracy: 0.7996 - val_loss: 0.7034
Epoch 5/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 137s 132ms/step - accuracy: 0.9797 - loss: 0.0598 - val_accuracy: 0.8244 - val_loss: 0.7100
Epoch 6/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 82s 132ms/step - accuracy: 0.9880 - loss: 0.0344 - val_accuracy: 0.8170 - val_loss: 0.8861
Epoch 7/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 83s 134ms/step - accuracy: 0.9871 - loss: 0.0401 - val_accuracy: 0.8210 - val_loss: 0.8887
Epoch 8/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 145s 139ms/step - accuracy: 0.9935 - los

In [47]:
preds=model.predict(padded_test_seq)

782/782 ━━━━━━━━━━━━━━━━━━━━ 38s 48ms/step


In [48]:
preds

array([[9.9036956e-01],
       [9.9989271e-01],
       [5.1233019e-05],
       ...,
       [1.4286717e-05],
       [9.9999201e-01],
       [9.9999279e-01]], dtype=float32)